In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import torch.nn as nn
import time
import os
from torch.utils.data import Subset
import random
import argparse

def get_random_subset_indices(num_samples, dataset_size):
    return random.sample(range(dataset_size), num_samples)


def create_dataset():
     # Data augmentation and normalization for training
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    

    train_dataset = torchvision.datasets.ImageFolder(os.path.join("/lus/eagle/projects/datascience/ImageNet/ILSVRC/Data/CLS-LOC", "train"), transform=train_transform)

    num_samples = 10000

    train_indices = get_random_subset_indices(num_samples, len(train_dataset))
    small_train_dataset = Subset(train_dataset, train_indices)
    return small_train_dataset

def build_model(GPU_selection):
    model = resnet50(weights=None)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model = nn.DataParallel(model,device_ids=GPU_selection)
    return model, criterion, optimizer, device

def create_dataLoader(batch_size=128, workers=4):
    return DataLoader(create_dataset(), batch_size=batch_size, num_workers=workers, shuffle=True, pin_memory=True)

# def select_device(selected_gpus):
#      os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, selected_gpus))

def train_one_epoch(model, criterion, optimizer, data_loader, device):
    model.train()
    
    total_images = 0
    start_time = time.time()
    start_time_dataLoad = time.time()
    end_time_dataLoad = 0
    for i, (inputs, labels) in enumerate(data_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_images += inputs.size(0)

    end_time = time.time()
    images_per_second = total_images / (end_time - start_time)
    dataLoad_time = end_time_dataLoad - start_time_dataLoad
    return int(images_per_second), end_time-start_time

def train(batch_size=256, GPU_selection=[0, 1], epoch=5, num_workers=8):
    num_epochs = epoch  # Adjust this value according to your needs
    train_loader = create_dataLoader(batch_size, num_workers)
    model, criterion, optimizer, device = build_model(GPU_selection)

    for epoch in range(num_epochs):
        images_per_second, epoch_duration = train_one_epoch(model, criterion, optimizer, train_loader, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Duration: {epoch_duration:.2f}s, Images/s: {images_per_second}")


            
# def main(args):
#     batch_size = args.batch_size
#     num_workers = args.number_worker
#     GPU_selection = [int(gpu) for gpu in args.GPU_selection.split(",")]

#     train(batch_size=batch_size, GPU_selection=GPU_selection, num_workers=num_workers)

# # if __name__ == "__main__":
# #     parser = argparse.ArgumentParser(description="Train ResNet-50 on ImageNet")
# #     parser.add_argument("--batch_size", type=int, default=256, help="Batch size for training (default: 256)")
# #     parser.add_argument("--number_worker", type=int, default=8, help="Number of workers for data loading (default: 8)")
# #     parser.add_argument("--GPU_selection", type=str, default="0,1", help="Comma-separated list of GPU indices to use (default: 0,1)")

# #     args = parser.parse_args()
# #     main(args)

In [ ]:
train(batch_size=256, GPU_selection=[0], epoch=5, num_workers=4)

In [2]:
train(batch_size=256, GPU_selection=[0, 1], epoch=5, num_workers=4)

Epoch [1/5], Duration: 76.57s, Images/s: 130
Epoch [2/5], Duration: 9.91s, Images/s: 1008
Epoch [3/5], Duration: 9.29s, Images/s: 1076
Epoch [4/5], Duration: 9.46s, Images/s: 1056
Epoch [5/5], Duration: 9.52s, Images/s: 1050


In [3]:
train(batch_size=256, GPU_selection=[0,1,2], epoch=5, num_workers=4)

Epoch [1/5],Duration: 67.96s, Images/s: 147
Epoch [2/5],Duration: 9.82s, Images/s: 1018
Epoch [3/5],Duration: 9.12s, Images/s: 1096
Epoch [4/5],Duration: 9.20s, Images/s: 1087
Epoch [5/5],Duration: 9.17s, Images/s: 1090


In [2]:
train(batch_size=256, GPU_selection=[0, 1,2,3], epoch=5, num_workers=4)

Epoch [1/5],Duration: 68.42s, Images/s: 146
Epoch [2/5],Duration: 9.30s, Images/s: 1075
Epoch [3/5],Duration: 9.11s, Images/s: 1097
Epoch [4/5],Duration: 9.13s, Images/s: 1094
Epoch [5/5],Duration: 9.27s, Images/s: 1079


In [ ]:
train(batch_size=256, GPU_selection=[0, 1,2,3], epoch=100, num_workers=8)

Epoch [1/100],Duration: 37.56s, Images/s: 266
Epoch [2/100],Duration: 5.56s, Images/s: 1797
Epoch [3/100],Duration: 5.82s, Images/s: 1717
Epoch [4/100],Duration: 5.61s, Images/s: 1784
Epoch [5/100],Duration: 5.73s, Images/s: 1744
Epoch [6/100],Duration: 5.73s, Images/s: 1744
Epoch [7/100],Duration: 5.69s, Images/s: 1755
Epoch [8/100],Duration: 5.62s, Images/s: 1779
Epoch [9/100],Duration: 5.51s, Images/s: 1814
Epoch [10/100],Duration: 5.77s, Images/s: 1733
Epoch [11/100],Duration: 5.80s, Images/s: 1722
Epoch [12/100],Duration: 6.21s, Images/s: 1609
Epoch [13/100],Duration: 5.84s, Images/s: 1713
Epoch [14/100],Duration: 5.57s, Images/s: 1794
Epoch [15/100],Duration: 5.67s, Images/s: 1764
Epoch [16/100],Duration: 5.59s, Images/s: 1787
Epoch [17/100],Duration: 5.71s, Images/s: 1749
